In [ ]:
import BAC0
import time
import csv
import json
import requests
import traceback
import sqlite3
from math import isnan 
import pandas as pd
import datetime

In [ ]:
ip_range = "192.168.1.6/24"  # Adjust this to your network range
csv_filename = "bacnet_devices.csv"
device_point_list = "all_controllers_and_points.xlsx"

In [ ]:
df = pd.DataFrame(columns=['TimeStamp', 'Device_ID', 'Device_Add', 'TAG', 'Value'])

In [ ]:
conn = sqlite3.connect('C:\\Users\\PARROT\\Desktop\\BACnetData.db')
c = conn.cursor()
c.execute("CREATE TABLE IF NOT EXISTS BACnet_LOG (id integer PRIMARY KEY, TimeStamp TEXT, Device_ID TEXT, Device_Add TEXT, TAG TEXT, Value TEXT)")
conn.commit()

In [ ]:
# bacnet = BAC0.lite(ip = '192.168.1.6/24')

In [ ]:
config_obj = {
    "host_ip":"192.168.1.6/24",
    "find_devices":False,
    "devices":[
        { "ip":"0xc0a80104f8c6", "device_id":770174 },
        { "ip":"0xc0a80106d615", "device_id":3588107 }
    ],
    "mqtt":{
        "host":"nube-io.com",
        "port":1883,
        "base_topic":"TEST/"
    },
    "loop_time":30
}

In [ ]:
def get_bac0_device(device, network):
    # print("Get BAC0 device")
    name, vendor, address, device_id = device
    # print(name, " | ", vendor, " | ", address, " | ", device_id, " | ")
    return BAC0.device(address, device_id, network, poll=0,object_list=None)

In [ ]:
def make_excel(dfs):
    with pd.ExcelWriter(device_point_list) as writer:
        for k, v in dfs.items():
            v.to_excel(writer, sheet_name=k)

In [ ]:
def create_data(discovered_devices, network):
    devices = {}
    dataframes = {}
    for each in discovered_devices:
        name, vendor, address, device_id = each
        # try excep eventually as we may have some issues with werid devices
        if "TEC3000" in name:
            custom_obj_list = tec_short_point_list()
        else:
            custom_obj_list = None
        devices[name] = BAC0.device(address, device_id, network, poll=0, object_list=custom_obj_list)
        # While we are here, make a dataframe with device
        dataframes[name] = make_dataframe(devices[name])
        devices[name].disconnect()
    return  dataframes

In [ ]:
def make_dataframe(dev):
    lst = {}
    for each in dev.points:
        lst[each.properties.name] = {
            "value": each.lastValue,
            "units or states": each.properties.units_state,
            "description": each.properties.description,
            "object": "{}:{}".format(each.properties.type, each.properties.address),
        }
    df = pd.DataFrame.from_dict(lst, orient="index")
    return df

In [ ]:
def save_to_csv(devices, csv_filename):
    with open(csv_filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        # Write headers to the CSV file
        writer.writerow(["Device Name", "Device ID", "Device Address", "Manufacturer"])
        # Write device information to the CSV file
        for device in devices:
            writer.writerow([device[0], device[3], device[2], device[1]])

In [ ]:
def discover_bacnet_devices(ip_range):
    # Create a BACnet IP network scanner
    global bacnet 
    bacnet = BAC0.lite(ip_range)
    # Discover BACnet devices on the network
    bacnet.discover()
    devices = bacnet.devices
    print(devices)
    dataframes = create_data(devices, network=bacnet)
    make_excel(dataframes)
    return devices

In [ ]:
devices = discover_bacnet_devices(ip_range)
save_to_csv(devices, csv_filename)

In [ ]:
# discover_bacnet_devices(ip_range)

In [ ]:
# devices

In [ ]:
if config_obj['devices']:
    while True:
        start_time = time.time()
        try:
            for a_device in devices:                
                device = get_bac0_device(a_device, bacnet)
                if not(isinstance(device, BAC0.core.devices.Device.DeviceDisconnected)):
                    for point in device.points:
                        temp_df = {'TimeStamp': datetime.datetime.now().strftime("%D/%M/%Y %H:%M:%S"),
                                   'Device_ID': str(device.properties.device_id) , 
                                   'Device_Add': str(device.properties.address), 
                                   'TAG': str(point.properties.name), 
                                   'Value': str(point.lastValue)}
                        df = pd.concat([df, pd.DataFrame(temp_df, index=[1])], ignore_index=True)
                
                device.disconnect()
                    
                        
        except Exception as e:
            print(traceback.print_exc())
            
        df.to_sql('BACnet_LOG', conn, if_exists='append', index=False)
        df.drop(df.index, inplace=True)
        
        time_elapsed = time.time() - start_time
        if config_obj['loop_time'] - time_elapsed > 0:
            sleep_time = config_obj['loop_time'] - time_elapsed
        else: 
            sleep_time = 0

        print("---- Loop took %.2f seconds ----" % time_elapsed)
        print("--- Sleeping for %.2f seconds ---" % sleep_time)
        time.sleep(sleep_time)